In [1]:
from IPython.display import Image
from IPython.display import display, HTML
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from fboost import outlier_iqr, DataPreparator, FeatureBoosterRegressor
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder

from sklearn.linear_model import LinearRegression, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.inspection import permutation_importance
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from ffx import FFXRegressor
import tensorflow as tf
from tensorflow import keras

from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score


import warnings
warnings.filterwarnings('ignore')

# Set a seed value
seed_value= 0
# 1. Set `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)
# 2. Set `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)
# 3. Set `numpy` pseudo-random generator at a fixed value
np.random.seed(seed_value)

/home/sandra/deedevelopment.ai/resmol/venv/lib/python3.10/site-packages/shap/utils/_clustering.py:35: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def _pt_shuffle_rec(i, indexes, index_mask, partition_tree, M, pos):
/home/sandra/deedevelopment.ai/resmol/venv/lib/python3.10/site-packages/shap/utils/_clustering.py:54: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
 

In [2]:
data = pd.read_csv('../data/raw/alldata.csv')
data.columns = ['familia','indice_fam','subposicion_1','subposicion_2','subposicion_3','subposicion_4', 'energia']
mask = (data[['subposicion_1','subposicion_2','subposicion_3','subposicion_4']] == 0).sum(axis=1) > 1
lista_train = data.loc[mask,['subposicion_1','subposicion_2','subposicion_3','subposicion_4','energia']]
lista_test = data.loc[~mask,['subposicion_1','subposicion_2','subposicion_3','subposicion_4','energia']]

In [3]:
# Instantiate the OneHotEncoder
encoder = OneHotEncoder(sparse=False, categories='auto')

X_train = lista_train.drop(['energia'], axis=1)
X_test = lista_test.drop(['energia'], axis=1)

# Fit the encoder and transform the data for both train and test dataframes
encoded_train = encoder.fit_transform(X_train)
encoded_test = encoder.transform(X_test)

# Now, 'encoded_train' and 'encoded_test' are numpy arrays, we can convert them back to dataframes:
X_train_scaled = pd.DataFrame(encoded_train, columns=encoder.get_feature_names_out(X_train.columns))
X_test_scaled = pd.DataFrame(encoded_test, columns=encoder.get_feature_names_out(X_test.columns))


y_test = lista_test['energia']
y_train = lista_train['energia']

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

# Assume you have a DataFrame `df` with 'sentence' and 'score' columns
df = pd.read_csv('../data/raw/alldata.csv')
# Converting all columns to string
df = df.astype(str)

df2 = df.replace('0','zero')
df2 = df2.replace('1','one')
df2 = df2.replace('2','two')
df2 = df2.replace('3','three')
df2 = df2.replace('4','four')
df2 = df2.replace('5','five')
print(df2)

     familia indice_fam subposicion_1 subposicion_2 subposicion_3  \
0        one        one          zero          zero          zero   
1        one        two          zero          zero           one   
2        one      three           one          zero          zero   
3        one       four          zero           one          zero   
4        two        one          zero          zero          zero   
...      ...        ...           ...           ...           ...   
1283     342        two          five           two          five   
1284     343        one         three          four         three   
1285     343        two          four         three          four   
1286     344        one         three          five         three   
1287     344        two          five         three          five   

     subposicion_4    energia  
0              one  1.2901211  
1             zero  1.2901211  
2             zero  1.2901211  
3             zero  1.2901211  
4          

In [5]:
# Convert DataFrame where each column is a word to a series of strings
df2['sentence'] = df2.apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

# Split data into training and test sets
train_data, test_data = train_test_split(df2, test_size=0.2, random_state=42)

# Specify your dictionary of words (customize n here)
dictionary = ['zero', 'one', 'two', 'three', 'four', 'five']

# Initialize TfidfVectorizer with your dictionary
# TfidfVectorizer will convert sentences into vectors of length n
vectorizer = TfidfVectorizer(vocabulary=dictionary)

# Transform the sentences in train and test data into vectors
X_train = vectorizer.fit_transform(train_data['sentence']).toarray()
X_test = vectorizer.transform(test_data['sentence']).toarray()

# Get the scores
y_train = train_data['energia']
y_test = test_data['energia']



In [6]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

# Create the individual models
model1 = RandomForestRegressor(n_estimators=100, random_state=42)
model2 = GradientBoostingRegressor(n_estimators=100, random_state=42)
model3 = make_pipeline(PolynomialFeatures(22), LinearRegression())
model4 = LinearRegression()

# Create the ensemble model
ensemble = VotingRegressor([('rf', model1), ('gb', model2), ('poly', model3), ('lr', model4)])

# Fit the ensemble model
ensemble.fit(X_train, y_train)

# Now you can make predictions with the ensemble model
predictions = ensemble.predict(X_test)

In [7]:
# Evaluate and score the model
score = ensemble.score(X_test, y_test)
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print('Score (MSE): ', score)
print('Mean Squared Error (MSE): ', mse)
print('Root Mean Squared Error (RMSE): ', rmse)
print('Mean Absolute Error (MAE): ', mae)
print('R-squared Score (R^2): ', r2)



Score (MSE):  -3.5052415016803422e+22
Mean Squared Error (MSE):  7.579465354034085e+23
Root Mean Squared Error (RMSE):  870601249369.3129
Mean Absolute Error (MAE):  135513689701.41672
R-squared Score (R^2):  -3.5052415016803422e+22


In [8]:
import io
import csv
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd

# Assume you have a DataFrame `df` with 'sentence' and 'score' columns
df = pd.read_csv('../data/raw/alldata.csv')

mask = (data[['subposicion_1','subposicion_2','subposicion_3','subposicion_4']] == 0).sum(axis=1) > 1
lista_train = data.loc[mask,['subposicion_1','subposicion_2','subposicion_3','subposicion_4','energia']]
lista_test = data.loc[~mask,['subposicion_1','subposicion_2','subposicion_3','subposicion_4','energia']]

X_train = lista_train.drop(['energia'], axis=1)
X_test = lista_test.drop(['energia'], axis=1)

y_test = lista_test['energia']
y_train = lista_train['energia']
# Converting all columns to string
X_train = X_train.astype(str)

X_train = X_train.replace('0','zero')
X_train = X_train.replace('1','one')
X_train = X_train.replace('2','two')
X_train = X_train.replace('3','three')
X_train = X_train.replace('4','four')
X_train = X_train.replace('5','five')

# Convert DataFrame where each column is a word to a series of strings, excluding 'energia' column
X_train['sentence'] = X_train.apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

print(X_train)


     subposicion_1 subposicion_2 subposicion_3 subposicion_4  \
0             zero          zero          zero           one   
1             zero          zero           one          zero   
2              one          zero          zero          zero   
3             zero           one          zero          zero   
4             zero          zero          zero           two   
...            ...           ...           ...           ...   
1265         three          zero         three          zero   
1266          zero          four          zero          four   
1267          four          zero          four          zero   
1268          zero          five          zero          five   
1269          five          zero          five          zero   

                   sentence  
0        zero zero zero one  
1        zero zero one zero  
2        one zero zero zero  
3        zero one zero zero  
4        zero zero zero two  
...                     ...  
1265  three zero thre

In [9]:
NUM_WORDS = 1000
EMBEDDING_DIM = 2
MAXLEN = 4
PADDING = 'post'
OOV_TOKEN = "<OOV>"
TRAINING_SPLIT = .8

In [10]:
def remove_stopwords(sentence):
    """
    Removes a list of stopwords

    Args:
        sentence (string): sentence to remove the stopwords from

    Returns:
        sentence (string): lowercase sentence without the stopwords
    """
    # List of stopwords
    stopwords = ["a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]

    # Sentence converted to lowercase-only
    sentence = sentence.lower()

    words = sentence.split()
    no_words = [w for w in words if w not in stopwords]
    sentence = " ".join(no_words)

    return sentence


def parse_data_from_dataframe(input_data):
    """
    Extracts sentences and labels from a CSV file

    Args:
        input_data (string): name of the dataframe

    Returns:
        sentences, labels (list of string, list of string): tuple containing lists of sentences and labels
    """
    sentence_list = []
    label_list = []
    for index, row in input_data.iterrows():
        #label_list.append(row['energia'])
        sentence_ind = row['sentence']
        sentence_ind = remove_stopwords(sentence_ind)
        sentence_list.append(sentence_ind)

    #return sentence_list, label_list
    return sentence_list

In [11]:
# Test the functions
#sentences, labels_array = parse_data_from_dataframe(X_train)
sentences = parse_data_from_dataframe(X_train)
print(sentences)

['zero zero zero one', 'zero zero one zero', 'one zero zero zero', 'zero one zero zero', 'zero zero zero two', 'zero zero two zero', 'two zero zero zero', 'zero two zero zero', 'zero zero zero three', 'zero zero three zero', 'three zero zero zero', 'zero three zero zero', 'zero zero zero four', 'zero zero four zero', 'four zero zero zero', 'zero four zero zero', 'zero zero zero five', 'zero zero five zero', 'five zero zero zero', 'zero five zero zero', 'zero zero one two', 'zero zero two one', 'two one zero zero', 'one two zero zero', 'zero zero one three', 'zero zero three one', 'three one zero zero', 'one three zero zero', 'zero zero one four', 'zero zero four one', 'four one zero zero', 'one four zero zero', 'zero zero one five', 'zero zero five one', 'five one zero zero', 'one five zero zero', 'zero zero two three', 'zero zero three two', 'three two zero zero', 'two three zero zero', 'zero zero two four', 'zero zero four two', 'four two zero zero', 'two four zero zero', 'zero zero 

In [12]:
#labels = [float(x) for x in labels_array]
labels = np.array(y_train)

print(f"There are {len(sentences)} sentences in the dataset.\n")
print(f"First sentence has {len(sentences[0].split())} words (after removing stopwords).\n")
print(f"There are {len(labels)} labels in the dataset.\n")
print(f"The first 5 labels are {labels[:5]}")

There are 170 sentences in the dataset.

First sentence has 4 words (after removing stopwords).

There are 170 labels in the dataset.

The first 5 labels are [1.2901211 1.2901211 1.2901211 1.2901211 0.0606045]


In [13]:
# GRADED FUNCTIONS: train_val_split
def train_val_split(sentences, labels, training_split):
    """
    Splits the dataset into training and validation sets

    Args:
        sentences (list of string): lower-cased sentences without stopwords
        labels (list of string): list of labels
        training split (float): proportion of the dataset to convert to include in the train set

    Returns:
        train_sentences, validation_sentences, train_labels, validation_labels - lists containing the data splits
    """

    ### START CODE HERE

    # Compute the number of sentences that will be used for training (should be an integer)
    train_size = int(len(sentences)*training_split)

    # Split the sentences and labels into train/validation splits
    train_sentences = sentences[0:train_size]
    train_labels = labels[0:train_size]

    validation_sentences = sentences[train_size:]
    validation_labels = labels[train_size:]

    ### END CODE HERE

    return train_sentences, validation_sentences, train_labels, validation_labels

In [14]:
# Test your function
train_sentences, val_sentences, train_labels, val_labels = train_val_split(X_train, labels, TRAINING_SPLIT)

print(f"There are {len(train_sentences)} sentences for training.\n")
print(f"There are {len(train_labels)} labels for training.\n")
print(f"There are {len(val_sentences)} sentences for validation.\n")
print(f"There are {len(val_labels)} labels for validation.")

There are 136 sentences for training.

There are 136 labels for training.

There are 34 sentences for validation.

There are 34 labels for validation.


In [15]:
def fit_tokenizer(train_sentences, num_words, oov_token):
    """
    Instantiates the Tokenizer class on the training sentences

    Args:
        train_sentences (list of string): lower-cased sentences without stopwords to be used for training
        num_words (int) - number of words to keep when tokenizing
        oov_token (string) - symbol for the out-of-vocabulary token

    Returns:
        tokenizer (object): an instance of the Tokenizer class containing the word-index dictionary
    """

    ### START CODE HERE

    # Instantiate the Tokenizer class, passing in the correct values for num_words and oov_token
    tokenizer = Tokenizer(num_words = num_words, oov_token=oov_token)

    # Fit the tokenizer to the training sentences
    tokenizer.fit_on_texts(train_sentences)

    ### END CODE HERE

    return tokenizer

In [16]:
# Test your function
tokenizer = fit_tokenizer(train_sentences, NUM_WORDS, OOV_TOKEN)
word_index = tokenizer.word_index

print(f"Vocabulary contains {len(word_index)} words\n")
print("<OOV> token included in vocabulary" if "<OOV>" in word_index else "<OOV> token NOT included in vocabulary")

Vocabulary contains 7 words

<OOV> token included in vocabulary


In [17]:
def seq_and_pad(sentences, tokenizer, padding, maxlen):
    """
    Generates an array of token sequences and pads them to the same length

    Args:
        sentences (list of string): list of sentences to tokenize and pad
        tokenizer (object): Tokenizer instance containing the word-index dictionary
        padding (string): type of padding to use
        maxlen (int): maximum length of the token sequence

    Returns:
        padded_sequences (array of int): tokenized sentences padded to the same length
    """

    ### START CODE HERE

    # Convert sentences to sequences
    sequences = tokenizer.texts_to_sequences(sentences)

    # Pad the sequences using the correct padding and maxlen
    padded_sequences = pad_sequences(sequences, maxlen=maxlen, padding=padding)

    ### END CODE HERE

    return padded_sequences

In [18]:
# Test your function
train_padded_seq = seq_and_pad(train_sentences, tokenizer, PADDING, MAXLEN)
val_padded_seq = seq_and_pad(val_sentences, tokenizer, PADDING, MAXLEN)

print(f"Padded training sequences have shape: {train_padded_seq.shape}\n")
print(f"Padded validation sequences have shape: {val_padded_seq.shape}")

Padded training sequences have shape: (5, 4)

Padded validation sequences have shape: (5, 4)


In [19]:
# GRADED FUNCTION: tokenize_labels
def tokenize_labels(all_labels, split_labels):
    """
    Tokenizes the labels

    Args:
        all_labels (list of string): labels to generate the word-index from
        split_labels (list of string): labels to tokenize

    Returns:
        label_seq_np (array of int): tokenized labels
    """

    ### START CODE HERE

    # Instantiate the Tokenizer (no additional arguments needed)
    label_tokenizer = Tokenizer()

    # Fit the tokenizer on all the labels
    label_tokenizer.fit_on_texts(all_labels)

    # Convert labels to sequences
    label_seq = label_tokenizer.texts_to_sequences(split_labels)

    # Convert sequences to a numpy array and subtract 1 from each entry
    label_seq_np = np.array(label_seq) - 1

    ### END CODE HERE

    return label_seq_np

In [20]:
# Test your function
train_label_seq = np.array(train_labels)
val_label_seq = np.array(val_labels)

print(f"First 5 labels of the training set should look like this:\n{train_label_seq[:5]}\n")
print(f"First 5 labels of the validation set should look like this:\n{val_label_seq[:5]}\n")
print(f"Tokenized labels of the training set have shape: {train_label_seq.shape}\n")
print(f"Tokenized labels of the validation set have shape: {val_label_seq.shape}\n")

First 5 labels of the training set should look like this:
[1.2901211 1.2901211 1.2901211 1.2901211 0.0606045]

First 5 labels of the validation set should look like this:
[0.8130417 0.8130417 0.8130417 0.8130417 4.4875204]

Tokenized labels of the training set have shape: (136,)

Tokenized labels of the validation set have shape: (34,)



In [21]:
def create_model(num_words, embedding_dim, maxlen):
    """
    Creates a text classifier model

    Args:
        num_words (int): size of the vocabulary for the Embedding layer input
        embedding_dim (int): dimensionality of the Embedding layer output
        maxlen (int): length of the input sequences

    Returns:
        model (tf.keras Model): the text classifier model
    """

    tf.random.set_seed(123)

    ### START CODE HERE

    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(num_words, embedding_dim, input_length=maxlen),
        tf.keras.layers.GlobalAveragePooling1D(),
        tf.keras.layers.Dense(6, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(loss='mean_squared_error',
                  optimizer='adam')

    ### END CODE HERE

    return model

In [22]:
model = create_model(NUM_WORDS, EMBEDDING_DIM, MAXLEN)

history = model.fit(train_padded_seq, train_label_seq, epochs=30, validation_data=(val_padded_seq, val_label_seq))

ValueError: Data cardinality is ambiguous:
  x sizes: 5
  y sizes: 136
Make sure all arrays contain the same number of samples.

In [ ]:


# Make predictions on the test set
predictions = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print('Mean Squared Error (MSE): ', mse)
print('Root Mean Squared Error (RMSE): ', rmse)
print('Mean Absolute Error (MAE): ', mae)
print('R-squared Score (R^2): ', r2)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.applications import MobileNetV2
from sklearn.metrics import mean_squared_error

# Assume you have a DataFrame `df` with 'sentence' and 'score' columns
df = pd.read_csv('../data/raw/alldata.csv')
# Converting all columns to string
df = df.astype(str)

df2 = df.replace('0','zero')
df2 = df2.replace('1','one')
df2 = df2.replace('2','two')
df2 = df2.replace('3','three')
df2 = df2.replace('4','four')
df2 = df2.replace('5','five')

# Convert DataFrame where each column is a word to a series of strings, excluding 'energia' column
df2['sentence'] = df2.drop(['familia','indice_fam','energia'], axis=1).apply(lambda row: ' '.join(row.values.astype(str)), axis=1)



# Split the data into training and test sets
train_data, test_data = train_test_split(df2, test_size=0.2, random_state=42)

# Parameters
max_length = 100  # Maximum length of sentences
embedding_dim = 100  # Dimensionality of word embeddings
vocab_size = 10000  # Maximum number of words in the vocabulary

# Tokenize the sentences
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(train_data['sentence'])
train_sequences = tokenizer.texts_to_sequences(train_data['sentence'])
test_sequences = tokenizer.texts_to_sequences(test_data['sentence'])

# Pad the sequences to have a consistent length
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding='post')
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding='post')

# Load pre-trained word embeddings (e.g., GloVe)
# Replace 'path_to_embeddings' with the actual path to your pre-trained embeddings file
embeddings_index = {}
with open('path_to_embeddings/glove.6B.100d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

# Create the embedding matrix
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

# Load pre-trained model
pretrained_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(max_length, embedding_dim))

# Build the model
model = Sequential()
model.add(pretrained_model)
model.add(LSTM(64))
model.add(Dense(1))

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')

# Train the model
model.fit(train_padded, train_data['score'], epochs=10, batch_size=32)

# Make predictions on the test set
predictions = model.predict(test_padded)

# Evaluate the model
mse = mean_squared_error(test_data['score'], predictions)
print(f"Mean Squared Error: {mse}")
